# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [4]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
#df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"Out of all the Bat-films, Batman Returns is my...",1
49996,"A decent sequel, but does not pack the punch o...",0
49997,"oh man, where to even begin.....<br /><br />dr...",0
49998,Many have disparaged Never Say Never Again bec...,1
49999,Sometimes a movie is so comprehensively awful ...,0


Let us shuffle the class labels.

In [5]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df.head()

,review,sentiment
11841,I have no clue as to what this was shot on but...,0
19602,"""The Seven-Ups"" seems like a replay of ""The Fr...",1
45519,"In Hollywood in the 1930's and 1940's, I think...",0
25747,There are worse ways to spend an evening than ...,1
42642,"""Elvira, Mistress Of The Dark"" is a sort of ""H...",1


<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [6]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Let's give it at try:

In [5]:
tokenizer('This :) is a <a> test! :-)</br>')

['test', ':)', ':)']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [7]:
def stream_docs(path):
    with open(path, 'r', encoding="utf8") as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [8]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"I have no clue as to what this was shot on but you can definitely tell that they had no budget. Bad acting, horrible cinematography, and lame plot and some decent special effects do not make a good movie. The WWF style cinemtography will make you cry...where\'s the tripod?! The filmakers aimed high, but sorely missed their mark."',
 0)

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [9]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return np.array(docs), np.array(y)

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [10]:
# from sklearn.feature_extraction.text import HashingVectorizer
# vect = HashingVectorizer(decode_error='ignore', 
#                          n_features=2**21,
#                          preprocessor=None, 
#                          tokenizer=tokenizer)

# Exercise 1: define features based on word embeddings (pre-trained word2vec / Glove/Fastext emebddings can be used)
# Define suitable d dimension, and sequence length
# https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/


doc_stream = stream_docs(path='shuffled_movie_data.csv')
X_train, y_train = get_minibatch(doc_stream, size=45000)
X_test, y_test = get_minibatch(doc_stream, size=5000)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.fit_transform(X_test)

X_train_vec.shape

(45000, 5000)

Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [15]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='shuffled_movie_data.csv')
# Exercise 2: Define at least a Three layer neural network. Define its structure (number of hidden neurons, etc)
# Define a nonlinear function for hidden layers.
# Define a suitable loss function for binary classification
# Implement the backpropagation algorithm for this structure
# Do not use Keras / Tensorflow /PyTorch etc. libraries
# Train the model using SGD


Red Neuronal de 3 capas:

In [51]:
D = X_train_vec.shape[1]
K = 2

# Numero de neuronas por capas
nn_layer_1 = D
nn_layer_2 = 200
nn_layer_3 = K

def initialize_parameters(nn_input_dim,nn_hdim,nn_output_dim):
    # First layer weights
    W1 = np.zeros((nn_input_dim, nn_hdim))
    
    # First layer bias
    b1 = np.zeros((nn_hdim, 1))
    
    # Second layer weights
    W2 = np.zeros((nn_hdim, nn_output_dim))
    
    # Second layer bias
    b2 = np.zeros((nn_output_dim, 1))
    
    
    # Package and return model
    model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
    return model

Non Linear function for hidden layers:

In [38]:
def relu(x):
    return np.maximum(x, 0)

In [39]:
def relu_deriv(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

Non Linear function for output layers:

Softmax:

In [40]:
def softmax(z):
    return np.exp(z-np.max(z))/np.sum(np.exp(z-np.max(z)), axis=0)

loss function for binary classification:

In [41]:
def compute_cost(y, y_hat):
    """
    y_hat is the predictions(1 x num_samples)
    y is labels (1 x num_samples)
    """
    m = y.shape[0]
    L = np.sum(-np.log(y_hat[y,range(m)])) / m
    return L

In [42]:
def delta_cross_entropy(y, y_hat):    
    ### START CODE HERE ### (≈ 4 lines of code)

    m = y.shape[0]
    y_hat[y,range(m)] -= 1
    y_hat = y_hat/m
    ### END CODE HERE ###
    
    return y_hat

In [52]:
def propagate_two_layer(W1, b1, W2, b2, X, y, use_reg=False, reg_lambda=0.01):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    W -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dW -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[0]
    num_examples = X.shape[0]
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 5 lines of code), one hidden layer, first layer with RELU, activation: softmax

#     X12 = np.maximum(np.dot(W1.T, X.T)+b1, 0)
    A0 = X.T
#     A1 = np.tanh(np.dot(W1.T, A0)+b1)
    A1 = relu_deriv(np.dot(W1.T, A0)+b1)
    A2 = softmax(np.dot(W2.T, A1)+b2)
    
    cost = compute_cost(y, y_hat=A2)
    
    ### END CODE HERE ###
    
    if use_reg:
        ### START CODE HERE ###  (≈ 2 add regularization here)
        
        reg_cost = 0.5 * reg_lambda * np.sum(W1*W1)+0.5 * reg_lambda * np.sum(W2*W2)
        cost = cost + reg_cost
        
        ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 3 lines of code)
    
    dz2 = delta_cross_entropy(y=y, y_hat=A2)
  
    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW2 = (1/m)*np.dot(A1, dz2.T)
    db2 = (1/m)*np.sum(dz2, axis=1)[:, np.newaxis]
    
    # next backprop into hidden layer
#     dhidden = ...
    dz1 = np.dot(W2, dz2)*relu_deriv(A1)
  
    # backprop the ReLU non-linearity (changed to Tanh)
    
    
    # finally backprop into parameters W1 and b1
    dW1 = (1/m)*np.dot(A0, dz1.T)
    db1 =  (1/m)*np.sum(dz1, axis=1)[:, np.newaxis]
    ### END CODE HERE ###
    
    if use_reg:
        ### START CODE HERE ###  (≈ 2 add backprop regularization here)
        dW1 += reg_lambda*W1 
        dW2 += reg_lambda*W2
        ### END CODE HERE ###

    assert(dW1.shape == W1.shape)
    assert(db1.dtype == float)
    assert(dW2.shape == W2.shape)
    assert(db2.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads, cost

In [53]:
def optimize_two_layer(W1, b1, W2, b2, X, y, num_iterations, learning_rate, use_reg = False, reg_lambda = 0.01, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    use_reg -- use regularization
    reg_lambda -- regularization weight
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for ii in range(num_iterations):
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        
        grads, cost = propagate_two_layer(W1, b1, W2, b2, X, y, use_reg=use_reg, reg_lambda=reg_lambda)

        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dW1 = grads["dW1"]
        db1 = grads["db1"]
        dW2 = grads["dW2"]
        db2 = grads["db2"]
        
        # update rule (≈ 4 lines of code)
        ### START CODE HERE ###
        W1 = W1 - learning_rate*dW1
        b1 = b1 - learning_rate*db1
        W2 = W2 - learning_rate*dW2
        b2 = b2 - learning_rate*db2
        ### END CODE HERE ###
        
        # Record the costs
        if ii % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 200 training iterations
        if print_cost and ii % 100 == 0:
            print ("Cost after iteration %i: %f" %(ii, cost))
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2}
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return params, grads, costs

In [54]:
def predict_two_layer(W1, b1, W2, b2, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    m = X.shape[0]
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    ### START CODE HERE ### (≈ 3 lines of code)

    A0 = X.T
#     A1 = np.tanh(np.dot(W1.T, X.T)+b1)
    A1 = relu(np.dot(W1.T, A0)+b1)
    A2 = softmax(np.dot(W2.T, A1)+b2)

    ### END CODE HERE ###
    
    predicted_class = np.argmax(A2, axis=0)
    
    assert(predicted_class.shape == (m,))
    
    return predicted_class

In [55]:
def train_two_layer(X_train, y_train, K=2, h_neurons=100, num_iterations=2000, learning_rate=0.5, use_reg=False, reg_lambda=0.01, init_type='random', print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    
    # initialize parameters with zeros (≈ 2 lines of code)
    model = initialize_parameters(D, h_neurons, K)
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize_two_layer(W1, b1, W2, b2, X_train, y_train, num_iterations, learning_rate, use_reg = use_reg, reg_lambda = reg_lambda, print_cost = print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Y_prediction_train = predict_two_layer(W1, b1, W2, b2, X_train)
    
    # Print train/test Errors
    print('training accuracy: %.2f' % (np.mean(Y_prediction_train == y)))
    
    d = {"costs": costs,
         "W1" : W1, 
         "b1" : b1,
         "W2" : W2, 
         "b2" : b2,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
d =  train_two_layer(X_train_vec, y_train, K=2,  h_neurons=5, num_iterations=900, learning_rate= 1e-0, use_reg=True, reg_lambda=1e-3, print_cost=True)

In [16]:
#import pyprind
#pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    #pbar.update()

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [17]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))
#Exercise 3: compare  with your Neural Network

Accuracy: 0.867


I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [18]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [21]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

['./clf.pkl']

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [22]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Writing tokenizer.py


In [23]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

['./tokenizer.pkl']

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [24]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [25]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

array([0])

In [26]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

array([1])